In [14]:
import pandas as pd
import random
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import calendar
import numpy as np

Downloaded data from: http://web.mta.info/developers/turnstile.html

Documentations at: http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt

Map of the MTA system: http://web.mta.info/maps/submap.html

In [15]:
df1 = pd.read_csv('turnstile_180526.txt')
df2 = pd.read_csv('turnstile_180602.txt')
df3 = pd.read_csv('turnstile_180609.txt')
df4 = pd.read_csv('turnstile_180616.txt')

In [16]:
df = pd.concat([df1, df2, df3, df4])

In [17]:
df.reset_index(inplace=True, drop=True)

Fix 'EXITS' column name (it has many spaces at the end of it)

In [18]:
# old_exit_name = df.columns[-1]
# df.rename(columns={df.columns[-1]: 'EXITS'}, inplace = True)
df.columns = df.columns.str.strip()

In [19]:
# df_mta.to_csv('turnstile_backup.csv')
df.to_csv('raw_turnstile_data.csv')

In [20]:
df['DATETIME'] = pd.to_datetime(df.DATE + ' ' + df.TIME, format='%m/%d/%Y %H:%M:%S')
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')

In [21]:
df['STATION_KEY'] = df['C/A'] + ' ' + df['UNIT'] + ' ' + df['STATION']

In [ ]:
# cols_to_drop = ['C/A','UNIT','STATION']
# cols_to_drop = ['C/A','UNIT']
# df.drop(cols_to_drop, axis=1, inplace=True)

Sort, Group by Station Key, then compute difference between 'ENTRIES'

In [23]:
df.sort_values(['STATION_KEY', 'SCP', 'DATETIME'], inplace=True)

In [26]:
df['ENTRY_DIFFS'] = df.groupby(['STATION_KEY','SCP'])['ENTRIES'].diff(periods=-1)*-1

In [27]:
df['EXIT_DIFFS'] = df.groupby(['STATION_KEY','SCP'])['EXITS'].diff(periods=-1)*-1

The majority of 'ENTRY_DIFFS' values are reasonable, but there are still some large negative and positive values still. For now, exclude them from our analysis. (There shouldn't, in fact, be any negative values.)

In [28]:
# 200,000 entries per day is a reasonable cut-off
clean_df = df[(df['ENTRY_DIFFS'] < 2E5) 
              & (df['ENTRY_DIFFS'] > 0) 
              & (df['EXIT_DIFFS'] < 2E5)
              & (df['EXIT_DIFFS'] > 0)]

In [29]:
clean_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,STATION_KEY,ENTRY_DIFFS,EXIT_DIFFS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-05-19,00:00:00,REGULAR,6625470,2245312,2018-05-19 00:00:00,A002 R051 59 ST,24.0,2.0
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-05-19,04:00:00,REGULAR,6625494,2245314,2018-05-19 04:00:00,A002 R051 59 ST,20.0,24.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-05-19,08:00:00,REGULAR,6625514,2245338,2018-05-19 08:00:00,A002 R051 59 ST,81.0,77.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-05-19,12:00:00,REGULAR,6625595,2245415,2018-05-19 12:00:00,A002 R051 59 ST,196.0,66.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-05-19,16:00:00,REGULAR,6625791,2245481,2018-05-19 16:00:00,A002 R051 59 ST,227.0,39.0


In [30]:
clean_df.to_csv('clean_turnstile_data.csv')